# SK-9-Building-CLR : Interoperabilite Python/.NET via pythonnet

**Navigation** : [Index](README.md) | [<< 08-MCP](08-SemanticKernel-MCP.ipynb) | [10-NotebookMaker >>](10-SemanticKernel-NotebookMaker.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Installer et configurer **pythonnet** pour l'interop Python/.NET
2. Charger des **DLL .NET** compilees depuis Python
3. Acceder aux **namespaces et classes C#** depuis Python
4. Appeler des **methodes asynchrones .NET** avec asyncio
5. Utiliser des **agents Semantic Kernel C#** depuis Python

### Prerequis

- Python 3.10+
- .NET SDK 9.0+ installe
- DLL `MyIA.AI.Notebooks.dll` compilee (`dotnet build`)
- Notebooks 01-08 completes (recommande)

### Duree estimee : 40 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Installation pythonnet | pip, CLR runtime |
| 2 | Chargement DLL | sys.path, clr.AddReference |
| 3 | Import classes C# | Assembly, GetType, namespaces |
| 4 | Execution agents | asyncio, nest_asyncio |

---

## Introduction

Ce notebook illustre comment, en Python, on peut :

1. **Installer et configurer** `pythonnet`
2. **Charger** la DLL .NET (compilee en C#) qui contient notre code d'agent
3. **Instancier** et **appeler** ces classes depuis Python

### Interpretation : Pourquoi l'interop Python/.NET ?

| Avantage | Description |
|----------|-------------|
| **Reutilisation** | Exploiter du code C# existant sans le reecrire |
| **Performance** | Beneficier de la compilation JIT .NET |
| **Ecosysteme** | Acceder aux bibliotheques .NET (Semantic Kernel C#, ML.NET) |
| **Hybride** | Combiner notebooks Python interactifs avec agents C# |

> **Note technique** : `pythonnet` utilise le CLR (Common Language Runtime) pour executer du code .NET depuis Python.

## Bloc 1 : Installation pythonnet & configuration

In [ ]:
# Installe pythonnet. À n'exécuter qu'une seule fois (enlevez --quiet si besoin)
%pip install pythonnet

In [ ]:
# On peut vérifier la version
import pkg_resources
print("pythonnet version :", pkg_resources.get_distribution("pythonnet").version)
print("Installation pythonnet : OK")

## Bloc 2 : Importation de la DLL .NET

Nous allons :  
1. **Modifier** le chemin Python (`sys.path`) pour inclure le dossier où se trouve la DLL .NET.  
2. **Charger** la DLL (`clr.AddReference`)  
3. **Importer** les namespaces/classes C# (ex. `MyIA.AI.Notebooks`).

Note : Adaptez `dll_path` si nécessaire.

In [ ]:
import sys
import clr
import os

# CORRECTION : Chemin absolu vers la DLL compilée
dll_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "bin", "Release", "net9.0"))
print(f"Chemin DLL calculé : {dll_path}")

# Vérification de l'existence du chemin
if not os.path.exists(dll_path):
    # Essayer le chemin Debug si Release n'existe pas
    dll_path_debug = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "bin", "Debug", "net9.0"))
    if os.path.exists(dll_path_debug):
        dll_path = dll_path_debug
        print(f"Utilisation du chemin Debug : {dll_path}")
    else:
        print(f"❌ Erreur : Aucun chemin DLL trouvé!")
        print(f"   - Testé Release : {dll_path}")
        print(f"   - Testé Debug : {dll_path_debug}")
        raise FileNotFoundError("DLL MyIA.AI.Notebooks.dll introuvable")

# On ajoute ce dossier dans sys.path
if dll_path not in sys.path:
    sys.path.append(dll_path)

print(f"✅ Chemin ajouté au sys.path : {dll_path}")

In [ ]:
# On charge la DLL (sans l'extension .dll)
try:
    clr.AddReference("MyIA.AI.Notebooks")
    print("✅ Référence à 'MyIA.AI.Notebooks.dll' ajoutée avec succès.")
except Exception as e:
    print(f"❌ Erreur lors du chargement de la DLL : {e}")
    # Lister les fichiers disponibles pour debug
    import os
    files = [f for f in os.listdir(dll_path) if f.endswith('.dll')]
    print(f"Fichiers .dll disponibles : {files}")
    raise

## Bloc 3 : Import des classes C# du namespace `MyIA.AI.Notebooks`

On suppose que les classes suivantes sont exposées :
- **`AutoInvokeSKAgentsNotebookUpdater`**  
- **`DisplayLogger`** (et son provider)  
- etc.

Ensuite, on pourra instancier et exécuter du code C# directement depuis Python.

In [ ]:
# Debug des assemblies chargées
import clr
from System import AppDomain

for asm in AppDomain.CurrentDomain.GetAssemblies():
    if asm.GetName().Name == "MyIA.AI.Notebooks":
        print(f"\n✅ Assembly trouvé: {asm.GetName().Name}")
        try:
            for t in asm.GetExportedTypes():
                print(f"  -> {t.FullName}")
        except Exception as e:
            print(f"  Erreur lors de l'énumération des types : {e}")
        break
else:
    print("❌ Assembly 'MyIA.AI.Notebooks' non trouvé")

In [ ]:
# Import correct pour pythonnet
import clr
from System.Reflection import Assembly
from System import AppDomain

# 1. Récupérer l'assembly déjà chargé
assembly = None
for asm in AppDomain.CurrentDomain.GetAssemblies():
    if asm.GetName().Name == "MyIA.AI.Notebooks":
        assembly = asm
        break

if assembly:
    print(f"✅ Assembly trouvé: {assembly.GetName().Name}")
    
    # 2. Lister tous les types pour debug (avec gestion d'erreur)
    print("Types disponibles dans l'assembly:")
    try:
        for t in assembly.GetTypes():
            print(f"  - {t.FullName}")
    except Exception as e:
        print(f"  ⚠️ Erreur lors de l'énumération des types: {e}")
        print(f"  ⚠️ Cela peut indiquer des dépendances .NET manquantes")
        print(f"  → Continuons avec la recherche de classes spécifiques...")
    
    # 3. Essayer plusieurs variantes du nom de classe
    type_names = [
        "MyIA.AI.Notebooks.GenAI.SemanticKernel.AutoInvokeSKAgentsNotebookUpdater",
        "MyIA.AI.Notebooks.AutoInvokeSKAgentsNotebookUpdater", 
        "AutoInvokeSKAgentsNotebookUpdater"
    ]
    
    AutoInvokeSKAgentsNotebookUpdater = None
    
    for type_name in type_names:
        updater_type = assembly.GetType(type_name)
        if updater_type:
            AutoInvokeSKAgentsNotebookUpdater = updater_type
            print(f"✅ Classe importée: {type_name}")
            print(f"   - Type: {AutoInvokeSKAgentsNotebookUpdater}")
            print(f"   - Nom complet: {AutoInvokeSKAgentsNotebookUpdater.FullName}")
            break
    
    if AutoInvokeSKAgentsNotebookUpdater is None:
        print("❌ Aucun type trouvé avec les noms testés")
        
else:
    print("❌ Assembly 'MyIA.AI.Notebooks' non trouvé")
    
    # Debug: lister toutes les assemblies chargées
    print("Assemblies chargées:")
    for asm in AppDomain.CurrentDomain.GetAssemblies():
        print(f"  - {asm.GetName().Name}")
    
    AutoInvokeSKAgentsNotebookUpdater = None

print("\n=== RÉSUMÉ IMPORT ===")
print(f"AutoInvokeSKAgentsNotebookUpdater = {AutoInvokeSKAgentsNotebookUpdater}")
print(f"Type: {type(AutoInvokeSKAgentsNotebookUpdater)}")

if AutoInvokeSKAgentsNotebookUpdater is not None:
    print("🎉 IMPORT .NET RÉUSSI !")
else:
    print("❌ Import .NET échoué - variable = None")

## Bloc 4 : Exemple d'utilisation

- On va créer une instance de `AutoInvokeSKAgentsNotebookUpdater`.
- On va définir un logger factice ou inexistant (selon la configuration).
- On va lancer la méthode `UpdateNotebookAsync()` pour lancer l'agent.

Note : Comme la méthode est asynchrone en C#, on l'appelle via `await` dans un contexte `asyncio` Python.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio

# Facultatif si vous avez un logger C# 
# from Microsoft.Extensions.Logging import LogLevel
# from MyIA.AI.Notebooks import DisplayLogger

async def run_agent_example():
    if AutoInvokeSKAgentsNotebookUpdater is None:
        print("❌ Classe AutoInvokeSKAgentsNotebookUpdater non disponible")
        return
    
    try:
        # logger = DisplayLogger("PyNotebookUpdater", LogLevel.Debug)  # Si vous avez un logger
        logger = None  # ou passer None si le constructeur l'accepte

        # Chemin où sera généré le notebook "cible" à manipuler
        notebook_path = r".\Workbook-Template-Python.ipynb"

        # On crée l'updater
        updater = AutoInvokeSKAgentsNotebookUpdater(notebook_path, logger)

        # On définit un template ou une instruction initiale
        # (Équivalent de: autoInvokeUpdater.SetStartingNotebookFromTemplate(...))
        # La méthode correspondante est "SetStartingNotebookFromTemplate" :
        updater.SetStartingNotebookFromTemplate("""
Créer un notebook Python, capable de requêter DBpedia via SPARQL, 
puis tracer un graphique Plotly. 
Ensuite, 
1) corriger d'éventuels bugs 
2) valider la sortie
""")

        # On exécute l'agent => C# : await autoInvokeUpdater.UpdateNotebookAsync()
        await updater.UpdateNotebookAsync()

        print("🎉 Agent SK terminé avec succès !")
        
    except Exception as e:
        print(f"❌ Erreur lors de l'exécution de l'agent : {e}")
        import traceback
        traceback.print_exc()

# Lancement
if AutoInvokeSKAgentsNotebookUpdater is not None:
    await run_agent_example()
else:
    print("❌ Impossible de lancer l'agent - classe non disponible")

## Explications

1. **`NotebookPath`** : Le fichier `.ipynb` que vous souhaitez générer / mettre à jour.
2. **`updater.SetStartingNotebookFromTemplate(...)`** : Injecte la consigne ou le template initial dans le notebook cible.
3. **`await updater.UpdateNotebookAsync()`** : Lance l'agent C# (AutoInvokeSKAgentsNotebookUpdater) qui va orchestrer Semantic Kernel pour incrémenter / améliorer le notebook.

Vous pouvez ensuite rouvrir le fichier `.ipynb` généré (dans un Jupyter Lab/VS Code) pour voir le résultat.

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **pythonnet** | Bridge Python/.NET | `import clr` |
| **clr.AddReference** | Chargement DLL | `clr.AddReference("MyAssembly")` |
| **Assembly.GetType** | Acces aux classes C# | `assembly.GetType("Namespace.Class")` |
| **nest_asyncio** | Asyncio dans notebooks | `nest_asyncio.apply()` |
| **await .NET async** | Appel methodes async C# | `await updater.UpdateNotebookAsync()` |

## Points cles a retenir

1. **pythonnet permet l'interop Python/.NET** - Chargez et utilisez des DLL C# directement
2. **Les assemblies .NET sont accessibles** - Reflection pour lister types et methodes
3. **Asyncio compatible** - Les methodes async C# s'appellent avec await
4. **Agents SK C# depuis Python** - Combinez le meilleur des deux mondes

## Architecture de l'interop

```
┌─────────────────────────────────────────────────────────────┐
│                      Python (Jupyter)                       │
│  ┌─────────────────────────────────────────────────────┐   │
│  │                    pythonnet (clr)                   │   │
│  │  ┌─────────────────────────────────────────────┐    │   │
│  │  │               CLR Runtime (.NET)            │    │   │
│  │  │  ┌─────────────────────────────────────┐   │    │   │
│  │  │  │     MyIA.AI.Notebooks.dll           │   │    │   │
│  │  │  │  - AutoInvokeSKAgentsNotebookUpdater│   │    │   │
│  │  │  │  - Semantic Kernel C#               │   │    │   │
│  │  │  └─────────────────────────────────────┘   │    │   │
│  │  └─────────────────────────────────────────────┘    │   │
│  └─────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
```

## Pour aller plus loin

| Notebook suivant | Contenu |
|------------------|---------|
| [10-NotebookMaker](10-SemanticKernel-NotebookMaker.ipynb) | Systeme 3-agents pour generation de notebooks |

---

**Navigation** : [Index](README.md) | [<< 08-MCP](08-SemanticKernel-MCP.ipynb) | [10-NotebookMaker >>](10-SemanticKernel-NotebookMaker.ipynb)